In [20]:
import pandas as pd
import scipy as sc
import numpy as np
from glob import glob
import parmed as pmd
import pytraj as pt
import subprocess as sp
from tqdm import tqdm_notebook

import os

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import colorConverter
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from constants import systems
from constants import guest_types
from constants import colors
import plotting

In [31]:
for system in systems:

    if not os.path.exists(f"systems/{system}/smirnoff/a000/vac.prmtop"):
        structure = pmd.load_file(f"systems/{system}/smirnoff/a000/smirnoff.prmtop")
        structure.strip(":WAT,:Na+,:Cl-")
        structure.save(f"systems/{system}/smirnoff/a000/vac.prmtop")
        print("Saved vacuum prmtop.")

    with open(f"systems/{system}/smirnoff/r014/flexibility.in", "w") as f:
        f.write("noexitonerror\n")
        f.write("parm vac.prmtop [simprmtop]\n")
        for count, file in enumerate(np.arange(0, 1000, 2.5)):
            if count % 2 == 0:
                frame = 1000
            else:
                frame = 2000
            f.write(f"trajin prod.{int(np.floor(file)):03d}.nc 1 {frame} 2000 parm [simprmtop]\n")
        
        f.write("parm ../hg.prmtop [refprmtop]\n")
        f.write("reference ../hg.inpcrd parm [refprmtop] [refstructure]\n")
        # f.write("strip @Pb\n")
        f.write("rmsd reference [refstructure] mass :MGO\n")
        f.write("trajout overlay.pdb pdb conect\n")
        f.write("run")
    
    with open(f"systems/{system}/smirnoff/r014/flexibility.sh", "w") as f:
        f.write("#!/usr/bin/bash\n")
        f.write("source /usr/local/programs/amber16gnu/amber.sh\n")
        f.write("cpptraj.OMP -i flexibility.in")

In [32]:
for system in tqdm_notebook(systems):

    print(system)
    sp.call("bash flexibility.sh", shell=True, cwd=f"systems/{system}/smirnoff/r014/")

a-bam-p
a-bam-s
a-but-p
a-but-s
a-cbu-p
a-chp-p
a-cbu-s
a-chp-s
a-cpe-p
a-coc-p
a-coc-s
a-cpe-s
a-hep-p
a-ham-s
a-ham-p
a-hep-s
a-hp6-p
a-hex-p
a-hex-s
a-hp6-s
a-hx2-p
a-hpa-s
a-hpa-p
a-hx2-s
a-mba-p
a-hx3-s
a-hx3-p
a-mba-s
a-mhp-p
a-mha-p
a-mha-s
a-mhp-s
a-nmh-p
a-nmb-p
a-nmb-s
a-nmh-s
a-oct-p
a-oam-p
a-oam-s
a-oct-s
a-pnt-p
a-pam-p
a-pam-s
a-pnt-s
b-ben-s
b-ben-p
b-cbu-p
b-cbu-s
b-chp-s
b-chp-p
b-coc-s
b-coc-p
b-cpe-s
b-cpe-p
b-ham-s
b-ham-p
b-hep-s
b-hep-p
b-hex-p
b-hex-s
b-m4c-s
b-m4c-p
b-m4t-p
b-m4t-s
b-mch-s
b-mha-s
b-mha-p
b-mch-p
b-mo3-s
b-mo4-p
b-mo4-s
b-mo3-p
b-mp3-s
b-mp4-s
b-mp4-p
b-mp3-p
b-oam-s
b-pb3-s
b-pb3-p
b-oam-p
b-pb4-s
b-pha-s
b-pb4-p
b-pha-p
b-pnt-s
b-pnt-p



In [27]:
for system in tqdm_notebook(systems):

    path = f"/gpfs/henrikse/bg2bg2-tip3p/{system}/r00/"
    ref_prmtop = f"/data/davids4/projects/smirnoff-host-guest-simulations-data/systems/{system}/smirnoff/hg.prmtop"
    ref_inpcrd = f"/data/davids4/projects/smirnoff-host-guest-simulations-data/systems/{system}/smirnoff/hg.inpcrd"
    
    if not os.path.exists(f"systems/{system}/bg2bg2-tip3p"):
        os.makedirs(f"systems/{system}/bg2bg2-tip3p")
    
    with open(f"systems/{system}/bg2bg2-tip3p/flexibility.in", "w") as f:
        f.write(f"parm {path}vac.topo [simprmtop]\n")
        
        
        for i in range(1, 201, 1):
            f.write(f"trajin {path}traj.{i:03d} 1 2000 1000 parm [simprmtop]\n")
        
        f.write(f"parm {ref_prmtop} [refprmtop]\n")
        f.write(f"reference {ref_inpcrd} parm [refprmtop] [refstructure]\n")
        f.write("rmsd reference [refstructure] mass :MGO\n")
        f.write("trajout overlay.pdb pdb conect\n")
        f.write("run")
    
    with open(f"systems/{system}/bg2bg2-tip3p/flexibility.sh", "w") as f:
        f.write("#!/usr/bin/bash\n")
        f.write("source /usr/local/programs/amber16gnu/amber.sh\n")
        f.write(f"cpptraj.OMP -i flexibility.in")
        
    sp.call(f"bash flexibility.sh", shell=True, cwd=f"systems/{system}/bg2bg2-tip3p/")

In [ ]:
# BGBG-TIP3P is missing. Ugh.

for system in systems:

    path = f"/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/{system}/r00/"
    ref_prmtop = f"/data/davids4/projects/smirnoff-host-guest-simulations-data/systems/{system}/smirnoff/hg.prmtop"
    ref_inpcrd = f"/data/davids4/projects/smirnoff-host-guest-simulations-data/systems/{system}/smirnoff/hg.inpcrd"
    
    with open(f"systems/{system}/bgbg-tip3p/free-cyclodextrin-rmsd.in", "w") as f:
        f.write(f"parm {path}vac.topo [simprmtop]\n")
        f.write(f"trajin {path}traj.* parm [simprmtop]\n")
        
        f.write(f"parm {ref_prmtop} [refprmtop]\n")
        f.write(f"reference {ref_inpcrd} parm [refprmtop] [refstructure]\n")
        f.write(f"rms reference [refstructure] :MGO&!@H= out free-cyclodextrin-rmsd.dat\n")
        f.write("run")
    
    with open(f"systems/{system}/bgbg-tip3p/free-cyclodextrin-rmsd.sh", "w") as f:
        f.write("#!/usr/bin/bash\n")
        f.write("source /usr/local/programs/amber16gnu/amber.sh\n")
        f.write(f"cpptraj.OMP -i free-cyclodextrin-rmsd.in")
        
    sp.call(f"bash free-cyclodextrin-rmsd.sh", shell=True, cwd=f"systems/{system}/bgbg-tip3p/")

In [28]:
for system in tqdm_notebook(systems):

    path = f"/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/{system}/r00/"
    ref_prmtop = f"/data/davids4/projects/smirnoff-host-guest-simulations-data/systems/{system}/smirnoff/hg.prmtop"
    ref_inpcrd = f"/data/davids4/projects/smirnoff-host-guest-simulations-data/systems/{system}/smirnoff/hg.inpcrd"
    
    if not os.path.exists(f"systems/{system}/bgbg-tip3p"):
        os.makedirs(f"systems/{system}/bgbg-tip3p")
    
    with open(f"systems/{system}/bgbg-tip3p/flexibility.in", "w") as f:
        f.write(f"parm {path}vac.topo [simprmtop]\n")
        
        
        for i in range(1, 201, 1):
            f.write(f"trajin {path}traj.{i:03d} 1 2000 1000 parm [simprmtop]\n")
        
        f.write(f"parm {ref_prmtop} [refprmtop]\n")
        f.write(f"reference {ref_inpcrd} parm [refprmtop] [refstructure]\n")
        f.write("rmsd reference [refstructure] mass :MGO\n")
        f.write("trajout overlay.pdb pdb conect\n")
        f.write("run")
    
    with open(f"systems/{system}/bgbg-tip3p/flexibility.sh", "w") as f:
        f.write("#!/usr/bin/bash\n")
        f.write("source /usr/local/programs/amber16gnu/amber.sh\n")
        f.write(f"cpptraj.OMP -i flexibility.in")
        
    sp.call(f"bash flexibility.sh", shell=True, cwd=f"systems/{system}/bgbg-tip3p/")

# Fix watery trajectories

In [ ]:
"""
parm smirnoff.prmtop
trajin prod.i.nc
strip :WAT,:Na+,:Cl-
trajout tmp.i.nc
run
quit
"""

mv tmp.i.nc prod.i.nc

BUT, this will fail...

In [15]:
for system in systems:
    windows = glob(f"systems/{system}/smirnoff/*")
    windows = [i for i in windows if os.path.isdir(i)]
    for window in windows:
        files = sorted(glob(f"{window}/prod*.nc"))
        for file in files:
            if os.stat(file).st_size > 10e6:
                print(f"Trimming... {file}")
                path = os.path.dirname(file)
                with open(f"{path}/trim.in", "w") as f:
                    f.write("parm smirnoff.prmtop\n")
                    f.write(f"trajin {os.path.basename(file)}\n")
                    f.write("strip :WAT,:Na+,:Cl-\n")
                    f.write(f"trajout {os.path.basename(file)}.tmp format netcdf\n")
                    f.write(f"run")
                with open(f"{path}/trim.sh", "w") as f:
                    f.write("#!/usr/bin/bash\n")
                    f.write("source /usr/local/programs/amber16gnu/amber.sh\n")
                    f.write(f"cpptraj -i trim.in\n")
                    f.write(f"mv {os.path.basename(file)} {os.path.basename(file)}.full\n")
                    f.write(f"mv {os.path.basename(file)}.tmp {os.path.basename(file)}\n")
    
                sp.call(f"bash trim.sh", shell=True, cwd=f"{path}")

Trimming... systems/a-bam-p/smirnoff/a000/prod.001.nc
Trimming... systems/a-bam-p/smirnoff/a000/prod.003.nc
Trimming... systems/a-bam-p/smirnoff/a000/prod.004.nc
Trimming... systems/a-bam-p/smirnoff/a000/prod.006.nc
Trimming... systems/a-bam-p/smirnoff/a000/prod.008.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.000.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.001.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.002.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.003.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.004.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.005.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.006.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.007.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.008.nc
Trimming... systems/a-bam-p/smirnoff/a001/prod.009.nc
Trimming... systems/a-bam-p/smirnoff/a002/prod.000.nc
Trimming... systems/a-bam-p/smirnoff/a002/prod.001.nc
Trimming... systems/a-bam-p/smirnoff/a002/prod.002.nc
Trimming... systems/a-bam-p/

In [12]:
for system in systems:
    windows = glob(f"systems/{system}/smirnoff/*")
    windows = [i for i in windows if os.path.isdir(i)]
    for window in windows:
        files = sorted(glob(f"{window}/prod*.nc.full"))
        for file in files:
            print(file)
            print(file[:-5])
            sp.call(f"mv {file} {file[:-5]}", shell=True)

systems/a-bam-p/smirnoff/a000/prod.001.nc.full
systems/a-bam-p/smirnoff/a000/prod.001.nc
systems/a-bam-p/smirnoff/a000/prod.003.nc.full
systems/a-bam-p/smirnoff/a000/prod.003.nc
systems/a-bam-p/smirnoff/a000/prod.004.nc.full
systems/a-bam-p/smirnoff/a000/prod.004.nc
systems/a-bam-p/smirnoff/a000/prod.006.nc.full
systems/a-bam-p/smirnoff/a000/prod.006.nc
systems/a-bam-p/smirnoff/a000/prod.008.nc.full
systems/a-bam-p/smirnoff/a000/prod.008.nc
systems/a-bam-p/smirnoff/a000/prod.009.nc.full
systems/a-bam-p/smirnoff/a000/prod.009.nc
systems/a-bam-p/smirnoff/a000/prod.010.nc.full
systems/a-bam-p/smirnoff/a000/prod.010.nc
systems/a-bam-p/smirnoff/a001/prod.000.nc.full
systems/a-bam-p/smirnoff/a001/prod.000.nc
systems/a-bam-p/smirnoff/a001/prod.001.nc.full
systems/a-bam-p/smirnoff/a001/prod.001.nc
systems/a-bam-p/smirnoff/a001/prod.002.nc.full
systems/a-bam-p/smirnoff/a001/prod.002.nc
systems/a-bam-p/smirnoff/a001/prod.003.nc.full
systems/a-bam-p/smirnoff/a001/prod.003.nc
systems/a-bam-p/smirn